# Decision Trees for Classification Part 2

Decision Tree is classification algorithm to build a model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore")

# fix_yahoo_finance is used to fetch data 
import fix_yahoo_finance as yf
yf.pdr_override()

In [2]:
# input
symbol = 'AMD'
start = '2014-01-01'
end = '2018-08-27'

# Read data 
dataset = yf.download(symbol,start,end)

# View Columns
dataset.head()

[*********************100%***********************]  1 of 1 downloaded


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2014-01-02,3.85,3.98,3.84,3.95,3.95,20548400
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700


In [3]:
# Create more data
dataset['Increase_Decrease'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],1,0)
dataset['Buy_Sell_on_Open'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],1,-1)
dataset['Buy_Sell'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],1,-1)
dataset['Return'] = dataset['Adj Close'].pct_change()
dataset = dataset.dropna()
dataset['Up_Down'] = np.where(dataset['Return'].shift(-1) > dataset['Return'],'Up','Down')
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Return,Up_Down
Date,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658,Up
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500,Down
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,-1,0.012107,Down
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,-1,-1,0.000000,Down
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,-1,1,-0.021531,Up


In [4]:
# Create more data
dataset['Open_N'] = np.where(dataset['Open'].shift(-1) > dataset['Open'],'Up','Down')
dataset['High_N'] = np.where(dataset['High'].shift(-1) > dataset['High'],'Up','Down')
dataset['Low_N'] = np.where(dataset['Low'].shift(-1) > dataset['Low'],'Up','Down')
dataset['Close_N'] = np.where(dataset['Adj Close'].shift(-1) > dataset['Adj Close'],'Up','Down')
dataset['Volume_N'] = np.where(dataset['Volume'].shift(-1) > dataset['Volume'],'Positive','Negative')
dataset.head()

,Open,High,Low,Close,Adj Close,Volume,Increase_Decrease,Buy_Sell_on_Open,Buy_Sell,Return,Up_Down,Open_N,High_N,Low_N,Close_N,Volume_N
Date,,,,,,,,,,,,,,,,
2014-01-03,3.98,4.00,3.88,4.00,4.00,22887200,1,1,1,0.012658,Up,Up,Up,Up,Up,Positive
2014-01-06,4.01,4.18,3.99,4.13,4.13,42398300,1,1,1,0.032500,Down,Up,Up,Up,Up,Positive
2014-01-07,4.19,4.25,4.11,4.18,4.18,42932100,0,1,-1,0.012107,Down,Up,Up,Up,Down,Negative
2014-01-08,4.23,4.26,4.14,4.18,4.18,30678700,0,-1,-1,0.000000,Down,Down,Down,Down,Down,Negative
2014-01-09,4.20,4.23,4.05,4.09,4.09,30667600,0,-1,1,-0.021531,Up,Down,Down,Up,Up,Negative


In [5]:
dataset.shape

(1171, 16)

In [6]:
X = dataset[['Open', 'Open_N', 'Volume_N']].values
y = dataset['Up_Down']

In [7]:
from sklearn import preprocessing
le_Open = preprocessing.LabelEncoder()
le_Open.fit(['Up','Down'])
X[:,1] = le_Open.transform(X[:,1]) 

le_Volume = preprocessing.LabelEncoder()
le_Volume.fit(['Positive', 'Negative'])
X[:,2] = le_Volume.transform(X[:,2]) 

In [8]:
from sklearn.model_selection import train_test_split  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)

In [9]:
from sklearn.tree import DecisionTreeClassifier  
classifier = DecisionTreeClassifier()  
classifier.fit(X_train, y_train)

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [10]:
# Modeling
Up_Down_Tree = DecisionTreeClassifier(criterion="entropy", max_depth = 4)
Up_Down_Tree

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [11]:
Up_Down_Tree.fit(X_train,y_train)

DecisionTreeClassifier(class_weight=None, criterion='entropy', max_depth=4,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [12]:
# Prediction
predTree = Up_Down_Tree.predict(X_test)

In [16]:
print(predTree[0:5])
print(y_test[0:5])

['Down' 'Up' 'Down' 'Up' 'Down']
Date
2018-01-23    Down
2016-07-27      Up
2015-10-06    Down
2016-03-09      Up
2014-05-06      Up
Name: Up_Down, dtype: object


In [17]:
# Evaluation
from sklearn import metrics
print("DecisionTrees's Accuracy: ", metrics.accuracy_score(y_test, predTree))

DecisionTrees's Accuracy:  0.6


In [19]:
# Accuracy Score without Sklearn
boolian = (y_test==predTree)
accuracy = sum(boolian)/y_test.size
accuracy

0.6